# Цель исследования

Проанализировать базу данных для формулирования ценностного предложения для нового продукта.

## Ход исследования:
  * Знакомство с данными 
     * Подключение к базе
     * Исследование таблиц
  * Исследовательский анализ
     * Количество книг после 01.01.2000
     * Количество обзоров и среднняя оценка для книг
     * Самое плодовитое издательство
     * Автор с самой высокой оценкой книг
     * Среднее количество обзоров активных пользователей
  * Вывод

# Знакомство с данными

In [1]:
import pandas as pd
from sqlalchemy import create_engine

## Подключение к базе данных

In [2]:
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Исследование таблиц

In [3]:
data = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for table in data:
    query = ''' SELECT * FROM {}; 
    '''.format(table) 

    display(pd.io.sql.read_sql(query, con = engine).head())
    print(pd.io.sql.read_sql(query, con = engine).info())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB
None


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB
None


**Таблицы отображаются корректно, пропусков нет в данных нет.** 

# Исследовательский анализ

## Книги после 1 января 2000 года

In [4]:
query = '''
SELECT COUNT (*)
FROM books 
WHERE publication_date > '2000-01-01'; 
''' 

pd.io.sql.read_sql(query, con = engine)

,count
0,819


**После 1 января 2000 года вышло 819 книг**

<a id='the_destination'></a>
## Количество обзоров и средняя оценка


In [5]:
query = '''
SELECT 
b.title AS book,
COUNT (DISTINCT r.review_id) AS count_reviews,
AVG (rat.rating) AS avg_rating
FROM
books AS b
LEFT JOIN reviews AS r ON b.book_id = r.book_id
LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
GROUP BY b.book_id
ORDER BY avg_rating DESC; 
''' 

pd.io.sql.read_sql(query, con = engine)

,book,count_reviews,avg_rating
0,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,Light in August,2,5.00
3,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,His Excellency: George Washington,2,2.00
997,Drowning Ruth,3,2.00
998,Junky,2,2.00


## Издательство, которое выпустило наибольшее число книг

In [6]:
query = '''
SELECT 
p.publisher AS publisher,
COUNT (b.title) AS book_count
FROM publishers AS p
INNER JOIN books AS b ON p.publisher_id = b.publisher_id 
WHERE b.num_pages > 50
GROUP BY publisher
ORDER BY book_count DESC
LIMIT 1; 
''' 

pd.io.sql.read_sql(query, con = engine)

,publisher,book_count
0,Penguin Books,42


**Издательство, выпустившее большее число книг - Penguin Books**

## Автор с самой высокой средней оценкой книг 

In [7]:
query = '''
SELECT 
a.author AS author,
AVG (r.rating) AS book_rating
FROM authors AS a
INNER JOIN books AS b ON a.author_id = b.author_id 
INNER JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY author
HAVING 
COUNT(r.rating_id)>=50
ORDER BY book_rating DESC
LIMIT 1
''' 

pd.io.sql.read_sql(query, con = engine)

,author,book_rating
0,Diana Gabaldon,4.3


**Автор, с самой высокой средней оценкой книг - Diana Gabaldon**	

## Среднее количество обзоров от пользователей

In [8]:
query = '''
SELECT 
AVG (DISTINCT (i.review_count)) AS review_avg
FROM (SELECT
username,
COUNT(review_id) AS review_count
FROM reviews
WHERE username IN (
  SELECT
  username
  FROM
  ratings
  GROUP BY
  username
  HAVING
  COUNT(rating_id) > 50)
GROUP BY username) AS i
''' 

pd.io.sql.read_sql(query, con = engine)

,review_avg
0,24.333333


**Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24**

# Вывод

Результатом проведенного анализа стало следушее:
  *  После 1 января 2000 года вышло 819 книг
  *  Составлена <a href='#the_destination'>таблица</a> количества обзоров и средней оценки для каждой книги
  *  Издательство, выпустившее большее число книг - Penguin Books (не считая брошюр)
  *  Автор, с самой высокой средней оценкой книг - Diana Gabaldon
  *  Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24